In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('/kaggle/input/ria-before-2025/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer


# MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
# model = AutoModelForCausalLM.from_pretrained(
#    MODEL_NAME,
#    device_map="cuda",
#    torch_dtype="auto",
# )

# input_text = "Кто сказал тебе, что нет на свете настоящей,"
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype="auto",
)


# Ограничимся первыми 10 новостями
news_samples = df["text"].head(10).tolist()

def generate_categories(news_text):
    prompt = f"""
    Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости (news) для российских читателей, создав две колонки: multi_labels и hier_label. Учитывай, что эти данные будут использоваться моделью Longformer для мультиклассовой классификации, поэтому количество уникальных классов должно быть ограничено.
    
    multi_labels: Создай список (list) из наиболее значимых категорий новости, в порядке убывания вероятности. Используй ограниченный набор категорий (не более 15). Выбирай наиболее общие и репрезентативные категории, которые охватывают основные темы новости. Избегай слишком узких или специфичных категорий.
    
    hier_label: Для первой категории из multi_labels создай двухуровневую иерархию (уровень1/уровень2) в виде списка. Уровень2 должен уточнять Уровень1 и быть более конкретным, чем любая категория из multi_labels, но оставаться в рамках разумного количества уникальных значений.
    
    На выходе должен быть JSON-объект с ключами 'multi_labels' и 'hier_label'.
    
    Текст новости: {news_text}
    """
    model.config.pad_token_id = tokenizer.pad_token_id
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
    outputs = model.generate(**input_ids, max_length=2048)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

results = [json.loads(generate_categories(news)) for news in news_samples]

# Записываем результат в DataFrame
df_result = pd.DataFrame({
    "index": range(10),
    "multi_labels": [res.get("multi_labels", []) for res in results],
    "hier_label": [res.get("hier_label", []) for res in results]
})


print(df_result)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]